In [2]:
#import libraries 
#psycopg2 creates the conncetions to sql dbs
import pandas as pd
import numpy as np
import psycopg2
import sys
from sqlalchemy import create_engine
import os
import subprocess
from datetime import datetime, timedelta
import tweepy

#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
postgres_user = os.environ['PSQL_USERNAME']
postgres_pw = os.environ['PSQL_PASSWORD']
postgres_url = os.environ['PSQL_URL']

redshift_user = os.environ['REDSHIFT_USERNAME']
redshift_pw = os.environ['REDSHIFT_PASSWORD']
redshift_url = os.environ['REDSHIFT_URL']

In [4]:
t_consumer_key = os.environ['TWT_CONSUMER_KEY']
t_consumer_secret = os.environ['TWT_CONSUMER_SECRET']
t_access_token = os.environ['TWT_ACCESS_TOKEN_KEY']
t_access_token_secret = os.environ['TWT_ACCESS_TOKEN_SECRET']

# authentication of consumer key and secret 
auth = tweepy.OAuthHandler(t_consumer_key, t_consumer_secret) 
    
# authentication of access token and secret 
auth.set_access_token(t_access_token, t_access_token_secret) 
api = tweepy.API(auth) 

def tweet(txtout):
    TARGET_DAY_FORMAT=datetime.today().strftime("%c")
    # update the status 
    api.update_status("status | "+TARGET_DAY_FORMAT+" "+txtout) 

In [5]:
tweet("Starting Additional Game Features")

In [6]:
#this block makes the connection to postgres db (tableau/follower)
conn_string = "host='"+postgres_url+"' dbname='experience' user='"+postgres_user+"' password='"+postgres_pw+"'"
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

In [7]:
#this block makes the connection to redshift (redluma)
red_engine = create_engine('postgresql://'+redshift_user+':'+redshift_pw+'@'+redshift_url)
conn_red = red_engine.raw_connection()
cur = conn_red.cursor()

In [8]:
statement=""" 
    select gr.group_type_id, count(*) as groups
    from "group" gr
    where group_type_id in (1,2,3,4,5,8,9,15,41)
    --where group_type_id in (3)
    group by 1;
    """  
group_types = pd.read_sql(statement, conn)
group_types_list = group_types['group_type_id'].tolist()

In [9]:
fuzzy = pd.read_sql("select distinct * from reporting.away_team_lookup_fuzzy;", conn_red)

column_indices = [1,3,4]
new_names_fuzzy = ['fixed_name','away_group_id','match_confidence']
old_names = fuzzy.columns[column_indices]

fuzzy_away=fuzzy.rename(columns=dict(zip(old_names, new_names_fuzzy)))

In [10]:
venue = pd.read_sql("select distinct * from venue;", conn)

column_indices = [2,4,5,8,9]
new_names_home = ['home_group_id','home_lat','home_long','home_avg_attendance','home_max_attendance']
new_names_away = ['away_group_id','away_lat','away_long','away_avg_attendance','away_max_attendance']
old_names = venue.columns[column_indices]

venue_away=venue.rename(columns=dict(zip(old_names, new_names_away)))
venue_home=venue.rename(columns=dict(zip(old_names, new_names_home)))

In [11]:
all_matches = pd.DataFrame(columns=[\
        'event_id',\
        'group_type_id',\
        'away_team',\
        'home_team_id',\
        'home_group_id',\
        'home_lat',\
        'home_long',\
        'home_venue_name',\
        'home_avg_attendance',\
        'home_max_attendance',\
        'fixed_name',\
        'away_group_id',\
        'match_confidence',\
        'away_lat',\
        'away_long',\
        'away_avg_attendance'])

for gtype in group_types_list:
    statement=""" 
    select distinct g.id, gr.group_type_id, g.away_team, g.home_team_id
    from game g
    left join "group" gr on g.home_team_id=gr.id
    where (g.game_start_time)>= '2017-01-01' and group_type_id={};
    """.format(str(gtype))  
    games = pd.read_sql(statement, conn)

    home_team_stats=pd.merge(games, venue_home, left_on='home_team_id',right_on='home_group_id', how='left')
    away_team_match=pd.merge(home_team_stats, fuzzy_away, on=['away_team','group_type_id'], how='inner')  
    away_team_stats=pd.merge(away_team_match, venue_away, left_on='away_group_id',right_on='away_group_id', how='left')  
    
    keep = away_team_stats[[\
        'id_x',\
        'group_type_id',\
        'away_team',\
        'home_team_id',\
        'home_group_id',\
        'home_lat',\
        'home_long',\
        'name_x',\
        'home_avg_attendance',\
        'home_max_attendance',\
        'fixed_name',\
        'away_group_id',\
        'match_confidence',\
        'away_lat',\
        'away_long',\
        'away_avg_attendance',\
        'away_max_attendance']]

    column_indices = [0,7]
    new_names_stats = ['event_id','home_venue_name']
    old_names = keep.columns[column_indices]

    keep_rename=keep.rename(columns=dict(zip(old_names, new_names_stats)))
    
    all_matches=all_matches.append(keep_rename, ignore_index=True)  
    
    

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [12]:
import geopy.distance
distances=[]

for index, row in all_matches.iterrows():
    away_lat=row['away_lat']
    away_long=row['away_long']
    coords_1 = (away_lat, away_long)
    home_lat=row['home_lat']
    home_long=row['home_long']
    coords_2 = (home_lat, home_long)
    if away_lat>0 and home_lat>0 and away_long<0 and home_long<0:
        distances.append(geopy.distance.distance(coords_1, coords_2).miles)
    else:
         distances.append(None)
            
all_matches['distance']=distances    
all_matches_with_distance=all_matches.dropna(subset=['distance'])
all_matches_with_distance.drop(all_matches_with_distance[all_matches_with_distance.distance < 5].index, inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
all_matches_stdv=all_matches_with_distance.groupby(['group_type_id'])['distance'].agg(np.std, ddof=0).to_frame()
all_matches_avg=all_matches_with_distance.groupby(['group_type_id'])['distance'].agg(np.average).to_frame()

all_matches_stdv.rename(columns = {'distance':'distance_stdev'}, inplace = True)
all_matches_avg.rename(columns = {'distance':'distance_avg'}, inplace = True)

all_matches = pd.merge(all_matches, all_matches_stdv,  how='left', on=['group_type_id'])
all_matches = pd.merge(all_matches, all_matches_avg,  how='left', on=['group_type_id'])

distance_z = []

for index, row in all_matches.iterrows():
    distance_z.append((row['distance']-row['distance_avg'])/row['distance_stdev'])

all_matches['distance_z'] = distance_z

In [14]:
all_matches['home_avg_attendance'] = all_matches['home_avg_attendance'].astype(float)
all_matches['home_max_attendance'] = all_matches['home_max_attendance'].astype(float)

all_matches_with_home_max_attendance=all_matches.dropna(subset=['home_max_attendance'])
all_matches_with_away_max_attendance=all_matches.dropna(subset=['away_max_attendance'])
all_matches_with_home_avg_attendance=all_matches.dropna(subset=['home_avg_attendance'])
all_matches_with_away_avg_attendance=all_matches.dropna(subset=['away_avg_attendance'])

all_matches_with_home_max_attendance.drop(all_matches_with_home_max_attendance[all_matches_with_home_max_attendance.home_max_attendance < 5].index, inplace=True)
all_matches_with_away_max_attendance.drop(all_matches_with_away_max_attendance[all_matches_with_away_max_attendance.away_max_attendance < 5].index, inplace=True)
all_matches_with_home_avg_attendance.drop(all_matches_with_home_avg_attendance[all_matches_with_home_avg_attendance.home_avg_attendance < 5].index, inplace=True)
all_matches_with_away_avg_attendance.drop(all_matches_with_away_avg_attendance[all_matches_with_away_avg_attendance.away_avg_attendance < 5].index, inplace=True)


In [15]:
all_matches_stdv=all_matches_with_home_max_attendance.groupby(['group_type_id'])['home_max_attendance'].agg(np.std, ddof=0).to_frame()
all_matches_avg=all_matches_with_home_max_attendance.groupby(['group_type_id'])['home_max_attendance'].agg(np.average).to_frame()

all_matches_stdv.rename(columns = {'home_max_attendance':'home_max_attendance_stdev'}, inplace = True)
all_matches_avg.rename(columns = {'home_max_attendance':'home_max_attendance_avg'}, inplace = True)

all_matches = pd.merge(all_matches, all_matches_stdv,  how='left', on=['group_type_id'])
all_matches = pd.merge(all_matches, all_matches_avg,  how='left', on=['group_type_id'])

home_max_attendance_z = []

for index, row in all_matches.iterrows():
    home_max_attendance_z.append((row['home_max_attendance']-row['home_max_attendance_avg'])/(row['home_max_attendance_stdev']+1))

all_matches['home_max_attendance_z'] = home_max_attendance_z

In [16]:
all_matches_stdv=all_matches_with_home_avg_attendance.groupby(['group_type_id'])['home_avg_attendance'].agg(np.std, ddof=0).to_frame()
all_matches_avg=all_matches_with_home_avg_attendance.groupby(['group_type_id'])['home_avg_attendance'].agg(np.average).to_frame()

all_matches_stdv.rename(columns = {'home_avg_attendance':'home_avg_attendance_stdev'}, inplace = True)
all_matches_avg.rename(columns = {'home_avg_attendance':'home_avg_attendance_avg'}, inplace = True)

all_matches = pd.merge(all_matches, all_matches_stdv,  how='left', on=['group_type_id'])
all_matches = pd.merge(all_matches, all_matches_avg,  how='left', on=['group_type_id'])

home_avg_attendance_z = []

for index, row in all_matches.iterrows():
    home_avg_attendance_z.append((row['home_avg_attendance']-row['home_avg_attendance_avg'])/(row['home_avg_attendance_stdev']+1))

all_matches['home_avg_attendance_z'] = home_avg_attendance_z

In [17]:
all_matches_stdv=all_matches_with_away_max_attendance.groupby(['group_type_id'])['away_max_attendance'].agg(np.std, ddof=0).to_frame()
all_matches_avg=all_matches_with_away_max_attendance.groupby(['group_type_id'])['away_max_attendance'].agg(np.average).to_frame()

all_matches_stdv.rename(columns = {'away_max_attendance':'away_max_attendance_stdev'}, inplace = True)
all_matches_avg.rename(columns = {'away_max_attendance':'away_max_attendance_avg'}, inplace = True)

all_matches = pd.merge(all_matches, all_matches_stdv,  how='left', on=['group_type_id'])
all_matches = pd.merge(all_matches, all_matches_avg,  how='left', on=['group_type_id'])

away_max_attendance_z = []

for index, row in all_matches.iterrows():
    away_max_attendance_z.append((row['away_max_attendance']-row['away_max_attendance_avg'])/(row['away_max_attendance_stdev']+1))

all_matches['away_max_attendance_z'] = away_max_attendance_z

In [18]:
all_matches_stdv=all_matches_with_away_avg_attendance.groupby(['group_type_id'])['away_avg_attendance'].agg(np.std, ddof=0).to_frame()
all_matches_avg=all_matches_with_away_avg_attendance.groupby(['group_type_id'])['away_avg_attendance'].agg(np.average).to_frame()

all_matches_stdv.rename(columns = {'away_avg_attendance':'away_avg_attendance_stdev'}, inplace = True)
all_matches_avg.rename(columns = {'away_avg_attendance':'away_avg_attendance_avg'}, inplace = True)

all_matches = pd.merge(all_matches, all_matches_stdv,  how='left', on=['group_type_id'])
all_matches = pd.merge(all_matches, all_matches_avg,  how='left', on=['group_type_id'])

away_avg_attendance_z = []

for index, row in all_matches.iterrows():
    away_avg_attendance_z.append((row['away_avg_attendance']-row['away_avg_attendance_avg'])/(row['away_avg_attendance_stdev']+1))

all_matches['away_avg_attendance_z'] = away_avg_attendance_z

In [19]:
events_additional_features=all_matches
events_additional_features.to_csv("assets/event_features.csv")

In [20]:
tweet("Additional Game Features Finished")